# Sistema especialista manutenção de hardware - Placa mãe

## 1 - Definindo os fatos

In [ ]:
from experta import * # type: ignore

class PlacaMae(Fact):
    """Fato que representa a quantidade de bipes emitidos pela placa-mãe no POST (Power-On Self Test)."""
    pass

## 2 - Sistema Especialista

As regras foram construídas com base na documentação de fabricantes de BIOS/placas-mãe (AMI e Gigabyte).  
Durante o processo de inicialização (**POST – Power-On Self Test**), a placa-mãe utiliza **códigos sonoros (bipes)** para indicar falhas de hardware antes mesmo da exibição de mensagens na tela.  

---

### 🔹 Regras
Cada regra associa um valor de bipes a um diagnóstico específico, de acordo com tabelas de fabricantes.  

---

### 🔹 Justificativa de cada regra

- **1 bipe → Erro de memória RAM**  
  A BIOS emite um bipe curto para indicar falha nos módulos de memória principal.

- **2 bipes → Erro de paridade da memória**  
  Indica que a memória não passou no teste de verificação de paridade.

- **3 bipes → Erro no endereço de 64 KB da memória básica**  
  Aponta problema em um endereço inicial da memória, fundamental para a inicialização.

- **4 bipes → Mau funcionamento do relógio em tempo real (RTC)**  
  O RTC é responsável pelo acompanhamento do tempo e armazenamento de informações de sistema; sua falha impede a configuração correta.

- **5 bipes → Erro na CPU**  
  Indica falha no processador, impossibilitando a execução de instruções básicas.

- **6 bipes → Erro no teclado**  
  O POST não conseguiu reconhecer ou inicializar o controlador do teclado.

- **7 bipes → Erro de interrupção da CPU**  
  Indica falhas no funcionamento interno do processador, especialmente em rotinas de interrupção.

- **8 bipes → Erro na placa de vídeo (GPU)**  
  A BIOS não conseguiu inicializar a controladora de vídeo, impedindo a exibição no monitor.

- **9 bipes → Novo erro de memória**  
  Outra forma de indicar falhas mais específicas na memória do sistema.

- **10 bipes → Erro na CMOS**  
  Indica falha na memória CMOS, que armazena as configurações básicas do hardware (setup da BIOS).

- **11 bipes → Mau funcionamento da memória cache da CPU**  
  Aponta defeito na memória cache do processador, essencial para desempenho e funcionamento correto.

Esse mapeamento foi implementado diretamente como regras (`@Rule`) no **motor de inferência**, tornando o sistema capaz de simular um diagnóstico automático.


In [ ]:
# cod de erros para placa mães Gigabyte e AMI
class DiagnosticoPlacaMae(KnowledgeEngine):
    """interpreta a quantidade de bipes e fornece o diagnóstico correspondente."""
    
    @Rule(PlacaMae(bipes=1))
    def erro_memoria(self):
        print("Diagnóstico: Memory Error. Erro de memória")

    @Rule(PlacaMae(bipes=2))
    def erro_paridade_memoria(self):
        print("Diagnóstico: Memory parity check error. Erro de checagem na paridade da memória")

    @Rule(PlacaMae(bipes=3))
    def erro_endereco_memoria(self):
        print("Diagnóstico: basic memory 64K address check error. Erro de checagem do endereço de 64k da memória básica")

    @Rule(PlacaMae(bipes=4))
    def erro_relogio(self):
        print("Diagnóstico: Real Time Clock malfunction. Mau funcionamento do relógio em tempo real")

    @Rule(PlacaMae(bipes=5))
    def erro_cpu(self):
        print("Diagnóstico: CPU error. Erro na CPU")

    @Rule(PlacaMae(bipes=6))
    def erro_teclado(self):
        print("Diagnóstico: Keyboard error. Erro no teclado")

    @Rule(PlacaMae(bipes=7))
    def erro_interrupcao_cpu(self):
        print("Diagnóstico: CPU interruption error. Erro de interrupção da CPU")

    @Rule(PlacaMae(bipes=8))
    def erro_placa_video(self):
        print("Diagnóstico: Graphic card error. Erro na placa de vídeo")

    @Rule(PlacaMae(bipes=9))
    def erro_memoria_novamente(self):
        print("Diagnóstico: Memory error. Erro de memória")

    @Rule(PlacaMae(bipes=10))
    def erro_cmos(self):
        print("Diagnóstico: CMOS error. Erro na CMOS")

    @Rule(PlacaMae(bipes=11))
    def erro_cache_cpu(self):
        print("Diagnóstico: CPU cache memory malfunction. Mau funcionamento na memória cache da CPU")

3 - Motor de inferencia

In [ ]:
engine = DiagnosticoPlacaMae()
engine.reset()

n_bipes = 0
while True:
    try:
        n_bipes = int(input("Digite o número de bipes da placa mãe: "))
        if n_bipes < 1 or n_bipes > 11:
            print("Os numeros de bipes válidos são entre 1 e 11. Tente novamente.")
            continue
        else:
            break
    except ValueError:
        print("Por favor, insira um número válido.")
        continue

engine.declare(PlacaMae(bipes=n_bipes))
engine.run()